In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
set_ID = 'Set001'

In [3]:
# this will take about 17 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 16.0 minutes to complete


In [56]:
# this will take about 1 minute
start = time.time()
gdf = match_taxlot(setID=set_ID, all_taxlot=all_taxlot, export=False)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 1.0 minutes to complete


In [57]:
gdf.shape

(2400, 31)

In [74]:
# this will take about 1 minute
start = time.time()
ngdf = check_corrected_data(setID=set_ID, all_taxlot=all_taxlot, export=False)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

AttributeError: 'DataFrame' object has no attribute 'ORTaxlot'

In [60]:
setID = 'Set001'

In [76]:
corrected = pd.read_excel(os.path.join(inpath, 'DSL data originals', 'Data corrections feedback to DSL', 
                           f'DSL Database corrections {setID}.xlsx'))
corrected.rename(columns={'trsqq': 'cor_trsqq',
                          'parcel_id':'cor_parcel_id'}, inplace=True)
res = review_with_lots(setID, all_taxlot)
gdf = res[0]
df_wlots = res[1]
IDs1 = check_duplicates(df_wlots.wetdet_delin_number.values)[0]
IDs2 = check_duplicates(corrected.wetdet_delin_number.values)[0]
comIDs = [ID for ID in IDs1 if ID in IDs2]
wdID_to_check = [wdID for wdID in df_wlots.wetdet_delin_number.values if wdID not in corrected.wetdet_delin_number.values]
cols1 = df_wlots.columns
cols2 = corrected.columns
comcols = [col for col in cols1 if col in cols2]
cols = [col for col in df_wlots.columns if col not in comcols]
cols.append('wetdet_delin_number')
df_wlots = df_wlots[cols]

In [77]:
if len(comIDs) > 0:
        df_wlots = df_wlots[~df_wlots.wetdet_delin_number.isin(comIDs)]
        corrected = corrected[~corrected.wetdet_delin_number.isin(comIDs)]
cor_df = corrected.merge(df_wlots, on = 'wetdet_delin_number')
cor_df = cor_df.drop(['trsqq', 'parcel_id'], axis=1)
cor_df.rename(columns={'cor_trsqq': 'trsqq', 
                      'cor_parcel_id': 'parcel_id',
                      'year': 'Year'}, inplace=True)
df_wlots_to_check = df_wlots[df_wlots.wetdet_delin_number.isin(wdID_to_check+comIDs)]
cor_df.loc[:, 'county'] = cor_df.county.apply(lambda x: string.capwords(x))

In [78]:
cor_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'matched ORTAXLOT#', 'Coord-Source', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [79]:
cor_df = reindex_data(cor_df)
cor_df_re = rematch_data(df=cor_df, setID=setID, all_taxlot=all_taxlot)

In [82]:
gdf.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'coord_src', 'doc_name',
       'doc_link', 'isbatfile', 'status_nm', 'receiveddt', 'responsedt',
       'reissuance', 'project_id', 'site_id', 'recordID', 'notes', 'lots',
       'lot', 'cnt_code', 'ORTaxlot', 'recyear', 'IDyear', 'missinglot',
       'record_ID', 'geometry'],
      dtype='object')

In [89]:
collist = list(gdf.columns)

In [90]:
collist

['wdID',
 'trsqq',
 'parcel_id',
 'loc_desc',
 'city',
 'county',
 'site_name',
 'site_desc',
 'latitude',
 'longitude',
 'coord_src',
 'doc_name',
 'doc_link',
 'isbatfile',
 'status_nm',
 'receiveddt',
 'responsedt',
 'reissuance',
 'project_id',
 'site_id',
 'recordID',
 'notes',
 'lots',
 'lot',
 'cnt_code',
 'ORTaxlot',
 'recyear',
 'IDyear',
 'missinglot',
 'record_ID',
 'geometry']

In [91]:
collist.remove('recordID')

In [88]:
cor_df_re.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'matched ORTAXLOT#', 'coord_src',
       'doc_name', 'doc_link', 'isbatfile', 'status_nm', 'receiveddt',
       'responsedt', 'reissuance', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'lots', 'lot', 'cnt_code',
       'ORTaxlot', 'geometry'],
      dtype='object')

In [92]:
ngdf = gdf.append(cor_df_re[collist])

In [40]:
wd_df = combine_wd_tables(set_ID)

In [41]:
wd_df.shape

(1420, 25)

In [42]:
len(gdf.record_ID.unique())

1308

In [51]:
def combined_reindexed_data(setID):
    frames = []
    files = list_files(os.path.join(wdpath, setID))
    # in case there are unidentified files
    files = [file for file in files if '~$' not in file]
    for file in files:
        wd_dt = clean_wd_table(setID, file = file)  
        frames.append(wd_dt)
    df = pd.concat(frames, ignore_index=True)
    wd_df = update_record(df, setID)
    return wd_df 

In [53]:
wd_df = combined_reindexed_data(set_ID)

In [55]:
def match_taxlot(setID, all_taxlot, export=False):
    all_txid = all_taxlot.ORTaxlot.unique()
    wd_df = combined_reindexed_data(setID)
    gdf = rematch_data(df=wd_df, setID=setID, all_taxlot=all_taxlot)
    if export:
        gdf[~gdf.geometry.isnull()].to_file(os.path.join(inpath + f'\\{outfolder}\\', f'matched_records_{setID}.shp'), 
                                                  driver='ESRI Shapefile')  
    return gdf

In [54]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'recordID',
       'notes', 'lots', 'lot', 'cnt_code', 'ORTaxlot', 'recyear', 'IDyear',
       'missinglot', 'record_ID'],
      dtype='object')

In [43]:
gdf.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'coord_src', 'doc_name',
       'doc_link', 'isbatfile', 'status_nm', 'receiveddt', 'responsedt',
       'reissuance', 'project_id', 'site_id', 'recordID', 'notes', 'lots',
       'lot', 'cnt_code', 'ORTaxlot', 'recyear', 'IDyear', 'missinglot',
       'record_ID', 'geometry'],
      dtype='object')

In [49]:
all_txid = all_taxlot.ORTaxlot.unique()
tocheck_txid = wd_df.ORTaxlot.unique()
found = [txid for txid in tocheck_txid if txid in all_txid]

AttributeError: 'DataFrame' object has no attribute 'ORTaxlot'

In [ ]:
taxlot_tocheck = all_taxlot[all_taxlot.ORTaxlot.isin(found)]
taxlot_tocheck = taxlot_tocheck.merge(tocheck_df, on='ORTaxlot', how='left')
taxlot_tocheck.loc[:, 'ydiff'] = taxlot_tocheck[['year', 'IDyear']].apply(lambda row: abs(int(row.year) - int(row.IDyear)), axis=1)
tdf = taxlot_tocheck.sort_values(by=['ORTaxlot', 'ydiff'])

KeyError: "['geometry'] not in index"

In [45]:
corrected = pd.read_excel(os.path.join(inpath, 'DSL data originals', 'Data corrections feedback to DSL', 
                           f'DSL Database corrections {set_ID}.xlsx'))
corrected.rename(columns={'trsqq': 'cor_trsqq', 
                      'parcel_id':'cor_parcel_id'}, inplace=True)
df_wlots = review_with_lots(gdf, set_ID, all_taxlot)[0]

In [46]:
df_wlots.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'coord_src', 'doc_name',
       'doc_link', 'isbatfile', 'status_nm', 'receiveddt', 'responsedt',
       'reissuance', 'project_id', 'site_id', 'recordID', 'notes', 'lots',
       'lot', 'cnt_code', 'ORTaxlot', 'recyear', 'IDyear', 'missinglot',
       'record_ID', 'geometry'],
      dtype='object')

In [24]:
frames = []
files = list_files(os.path.join(wdpath, set_ID))
# in case there are unidentified files
files = [file for file in files if '~$' not in file]
for file in files:
    wd_dt = clean_wd_table(set_ID, file = file)  
    frames.append(wd_dt)
df = pd.concat(frames, ignore_index=True)

C:\Users\clid1852\AppData\Local\Temp\ipykernel_18888\3501787811.py:98: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ndf.loc[selectedID, 'missinglot'] = ndf[selectedID].parcel_id.apply(lambda x: without_lots(x))


In [29]:
wd_df.shape

(2496, 30)

In [96]:
wd_df = combine_wd_tables(set_ID)

In [28]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'recordID',
       'notes', 'lots', 'lot', 'cnt_code', 'ORTaxlot', 'recyear', 'IDyear',
       'missinglot', 'record_ID'],
      dtype='object')

In [38]:
len(gdf.record_ID.unique())

1308

In [30]:
all_txid = all_taxlot.ORTaxlot.unique()

In [ ]:
df_ri = reindex_data(df)

In [3]:
#gdf = combine_merged_data(setID = set_ID)

In [31]:
#gdf = gpd.read_file(os.path.join(inpath + f'\\{outfolder}\\', f'matched_records_{set_ID}.shp'), driver='ESRI Shapefile')
gdf = gpd.read_file(os.path.join(inpath + f'\\{outfolder}\\', f'combined_records_in_{set_ID}.shp'), driver='ESRI Shapefile')

it took 17.0 minutes to complete


In [5]:
cor_df = check_corrected_data(gdf=gdf, setID=set_ID, all_taxlot=all_taxlot)[0]

In [6]:
comIDs = check_corrected_data(gdf=gdf, setID=set_ID, all_taxlot=all_taxlot)[1]

In [7]:
cor_df.shape

(23, 26)

In [54]:
wd_df.shape

(1420, 25)

In [9]:
df_wlots = review_with_lots(gdf, set_ID, all_taxlot)[0]

In [10]:
df_wlots_to_check = check_corrected_data(gdf=gdf, setID=set_ID, all_taxlot=all_taxlot)[2]

In [39]:
import string

In [41]:
cor_df.loc[:, 'county'] = cor_df.county.apply(lambda x: string.capwords(x))

In [45]:
cor_df_re = rematch_data(gdf=cor_df, setID=set_ID, all_taxlot=all_taxlot, new=True)

In [47]:
cor_df_re.shape

(25, 31)

In [48]:
cor_df_re.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'matched ORTAXLOT#', 'coord_src',
       'doc_name', 'doc_link', 'isbatfile', 'status_nm', 'receiveddt',
       'responsedt', 'reissuance', 'project_id', 'site_id', 'record_ID',
       'notes', 'Year', 'IDyear', 'missinglot', 'lots', 'lot', 'cnt_code',
       'ORTaxlot', 'geometry'],
      dtype='object')

In [49]:
gdf.columns

Index(['record_ID', 'wdID', 'parcel_id', 'ORTaxlot', 'notes', 'geometry'], dtype='object')

In [50]:
ngdf = gdf.append(cor_df_re[gdf.columns])

In [93]:
ngdf.shape

(2425, 31)

In [94]:
len(ngdf.record_ID.unique())

1331

In [53]:
ngdf.shape

(2407, 6)

In [52]:
ngdf.to_file(os.path.join(inpath + f'\\{outfolder}\\', f'more_combined_records_in_{set_ID}.shp'), driver='ESRI Shapefile')

In [55]:
len(ngdf.record_ID.unique())

1331

In [97]:
wd_df.shape[0] - len(ngdf.record_ID.unique())

89

In [98]:
len(df_wlots_to_check.record_ID.unique())

28

In [59]:
gdf1st = gpd.read_file(os.path.join(inpath + f'\\{outfolder}\\', f'matched_records_{set_ID}.shp'), driver='ESRI Shapefile')

In [99]:
missinglot = records_with_lots(gdf=gdf, setID=set_ID)

In [100]:
len(missinglot.record_ID.unique())

61

In [102]:
setgdf = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_{set_ID}_Combined')

In [67]:
df_wlots_to_check.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'year', 'IDyear', 'missinglot'],
      dtype='object')

In [69]:
setgdf.columns

Index(['Record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id',
       'address_location_desc', 'city', 'county', 'site_name', 'site_desc',
       'latitude', 'longitude', 'Coord_Source', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'lot_num', 'lot',
       'Match_Year', 'ORTAXLOT', 'Match_found', 'Manual_note', 'Shape_Length',
       'Shape_Area', 'Edits_Complete', 'matcher2', 'geometry'],
      dtype='object')

In [106]:
setgdf_s = setgdf[setgdf.Record_ID.isin(df_wlots_to_check.record_ID)]

In [107]:
setgdf_s.loc[:, 'record_ID'] = setgdf_s.loc[:, 'Record_ID'].astype(int)

In [108]:
rIDs = setgdf_s.Record_ID.unique()

In [109]:
df_wlots_to_check[df_wlots_to_check.record_ID.isin(rIDs)][['record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id']].merge(setgdf_s[['record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id']], on='record_ID')

,record_ID,wetdet_delin_number_x,trsqq_x,parcel_id_x,wetdet_delin_number_y,trsqq_y,parcel_id_y
0,83,WD2020-0185,05N10W014,501,WD2020-0185,05N10W14,501
1,176,WD2022-0187,06N10W22,8100,WD2022-0187,06N10W22AB,8100
2,182,WD2017-0016,28S15W36AD,1302,WD2017-0016,28S15W36AD,1302
3,223,WD2019-0122,26S14W02D,300,WD2019-0122,26S14W02D,300
4,235,WD2019-0338,24S13W34C,140p,WD2019-0338,24S13W34C,140p
5,237,WD2019-0338,24S13W00,200p,WD2019-0338,24S13W00,200p
6,416,WD2020-0067,17S12E10C,"400,500 all partial",WD2020-0067,17S12E10C,"400,500 all partial"
7,477,WD2020-0586,20S10E25BB,400p,WD2020-0586,20S10E25B,400p
8,484,WD2020-0668,18S12E08DA,"ROW, 6317,6303,3604",WD2020-0668,18S12E08DA,"ROW, 6317,6303,3604"
9,494,WD2021-0066,18S12W25B,400,WD2021-0066,18S12E25B,400
